New Journal Amps Extraction
===

This script processes the json amps in the new (2019) dataset to a CSV file containing the barebones interaction info.


In [13]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [14]:
import os
import re
import pandas as pd
import numpy as np

from collections import Counter
import sqlite3
from nltk import word_tokenize
from html.parser import HTMLParser
from tqdm import tqdm
import random
import pickle
import json

from datetime import datetime
from pprint import pprint

import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib
import pylab as pl
from IPython.core.display import display, HTML

In [15]:
from pathlib import Path
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = Path(git_root_dir[0].strip())
git_root_dir

PosixPath('/panfs/roc/groups/3/srivbane/levon003/repos/sna-social-support')

In [16]:
import sys
caringbridge_core_path = "/home/srivbane/levon003/repos/caringbridge_core"
sys.path.append(caringbridge_core_path)

In [17]:
import cbcore.data.paths as paths
import cbcore.data.dates as dates

In [18]:
raw_data_dir = paths.raw_data_2019_filepath

In [19]:
journal_filepath = os.path.join(raw_data_dir, "journal.json")

In [20]:
working_dir = "/home/srivbane/shared/caringbridge/data/projects/sna-social-support/csv_data"
assert os.path.exists(working_dir)

### Load and convert journal file

In [21]:
output_filepath = os.path.join(working_dir, "new_amps_metadata_raw.csv")

In [22]:
def extract_long(json_value):
    if type(json_value) == int:
        return json_value
    elif type(json_value) == str:
        return int(json_value)
    elif type(json_value) == dict:
        return int(json_value['$numberLong'])
    elif json_value == 0.0:  # special case for userId, which I assume is essentially "missing/unknown" or user deleted?
        return None
    elif type(json_value) == float:
        return int(json_value)
    else:
        raise ValueError(f"Type '{type(json_value)}' not implemented: {str(json_value)}")

In [23]:
with open(output_filepath, 'w') as outfile:
    with open(journal_filepath, encoding='utf-8') as infile:
        for line in tqdm(infile, total=19137078):
            journal = json.loads(line)
            
            if 'amps' not in journal:
                continue
            amps = journal['amps']
            if type(amps) != list:
                continue
                
            journal_oid = journal['_id']['$oid']
            site_id = extract_long(journal['siteId'])
            update_user_id = extract_long(journal['userId'])
            
            created_at = dates.get_date_from_json_value(journal['createdAt'])
            updated_at = dates.get_date_from_json_value(journal['updatedAt'])
            
            for amp in amps:
                amp_user_id = extract_long(amp)
                outfile.write(f"{amp_user_id},{site_id},{journal_oid},{update_user_id},{created_at},{updated_at}\n")

        

100%|██████████| 19137078/19137078 [56:16<00:00, 5668.18it/s]  
